In [1]:
# 라이브러리 설치
!pip install transformers accelerate bitsandbytes scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00


In [2]:
# 패키지 임포트
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import files
import pandas as pd

In [3]:
# 모델 로드
model_name = "beomi/KoAlpaca-Polyglot-5.8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# 프롬프트
def make_prompt(question):
    return f"""너는 사용자 질문을 아래 5개 카테고리 중 하나로 분류하는 분류기야.
0: 졸업요건
1: 학교 공지사항
2: 학사 일정
3: 식단 안내
4: 통학버스

예시:
Q: 오늘 학식 뭐야?
A: 3
Q: 수강 신청 언제야?
A: 2
Q: 통학버스 언제 와?
A: 4
Q: {question}
A:"""

In [5]:
# LLM 기반 분류 함수
def classify_question(question):
    prompt = make_prompt(question)
    output = generator(prompt, max_new_tokens=5, do_sample=False)[0]["generated_text"]
    result = output.strip().split("A:")[-1].strip().split()[0]
    return result if result in ['0', '1', '2', '3', '4'] else "예측 실패"

In [6]:
# 키워드 기반 + LLM 혼합 분류 함수
def keyword_or_llm_classify(question):
    q = question.lower()
    keyword_map = {
        0: ["졸업", "이수", "학점", "인증", "시험"],
        1: ["장학금", "공지", "예비군", "등록금", "공모전", "특강"],
        2: ["개강", "종강", "방학", "수강", "성적", "정정", "학사", "휴강"],
        3: [ "학식", "메뉴", "점심", "저녁", "식당", "다이어트", "건강", "식단"],
        4: ["버스", "통학", "정류장", "노선", "운행"]
    }
    for label, keywords in keyword_map.items():
        if any(kw in q for kw in keywords):
            return str(label)
    return classify_question(question)

In [14]:
# 데이터 업로드
file_path = "/content/drive/MyDrive/final_relabeled_queries_label1_2_swapped.json"

with open(file_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [19]:
dataset[:5]

[{'question': '제1학생회관 저녁 언제부터 해?', 'label': 3},
 {'question': '오늘 점심 학식 뭐야?', 'label': 3},
 {'question': '김치찌개 먹고 싶은데 1학에 이 메뉴가 있을까? ', 'label': 3},
 {'question': '이번 주 생활관 메뉴 중에 미역국 있어?', 'label': 3},
 {'question': '이번 주 메뉴 중에 잔치국수 있어?', 'label': 3}]

In [16]:
# 평가
y_true = []
y_pred = []

for sample in dataset:
    true_label = str(sample["label"])
    question = sample["question"]
    pred_label = keyword_or_llm_classify(question)
    y_true.append(true_label)
    y_pred.append(pred_label)

# 예측 실패 제거
filtered = [(yt, yp) for yt, yp in zip(y_true, y_pred) if yp != "예측 실패"]
y_true, y_pred = zip(*filtered)

# 성능 지표 출력
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

print("\n📊 평가 결과")
print(f"- Accuracy:  {accuracy:.4f}")
print(f"- Precision: {precision:.4f}")
print(f"- Recall:    {recall:.4f}")
print(f"- F1 Score:  {f1:.4f}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore


📊 평가 결과
- Accuracy:  0.9685
- Precision: 0.9691
- Recall:    0.9599
- F1 Score:  0.9636


In [18]:
# 사용자 입력에 대해 실시간 분류
while True:
    user_input = input("\n질문을 입력하세요 (종료하려면 'exit'): ")
    if user_input.strip().lower() == 'exit':
        break
    result = keyword_or_llm_classify(user_input)
    label_map = {
        "0": "졸업요건",
        "1": "학교 공지사항",
        "2": "학사 일정",
        "3": "식단 안내",
        "4": "통학버스",
        "예측 실패": "분류 실패"
    }
    print(f"예측 결과: {label_map.get(result, '오류')} ({result})")


질문을 입력하세요 (종료하려면 'exit'): 오늘 2학 점심 뭐야?
예측 결과: 식단 안내 (3)

질문을 입력하세요 (종료하려면 'exit'): 우리 몇 학저 들어야 해?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


예측 결과: 분류 실패 (예측 실패)

질문을 입력하세요 (종료하려면 'exit'): 졸업하려면 몇 학점 들어야해?
예측 결과: 졸업요건 (0)

질문을 입력하세요 (종료하려면 'exit'): 버스 어디서 타?
예측 결과: 통학버스 (4)

질문을 입력하세요 (종료하려면 'exit'): exit
